In [1]:
# load modules
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.graph_objs as go

In [2]:
# personal mapbox token
mapbox_access_token = "pk.eyJ1IjoiaHl2aCIsImEiOiJja3RrNWlzcGgwMDQzMm5vODYwcHJvcmRlIn0.QIsGMFNngsjk4GZD_Nou8g"

In [3]:
df = pd.read_csv("final_daft_listings.csv")

In [18]:
df2 = pd.read_csv("C:/Users/hyvhu/Desktop/Neuer Ordner/Planungsbueros_Deutschland.csv")

### Remove all listings without Latitude/Longitude

In [4]:
df[["Longitude", "Latitude"]].isnull().sum()

Longitude    1033
Latitude     1033
dtype: int64

In [5]:
df.columns

Index(['Address', 'Room', 'Bath', 'Property_type', 'Bedrooms_available',
       'Available_from', 'Available_for', 'Sharing_with', 'Owner_occupied',
       'Preferences', 'Date_entered/renewed', 'Property_views', 'Latitude',
       'Longitude', 'Pre_processed_desc', 'Desc_length', 'Price_per_month',
       'County', 'adjective_counts'],
      dtype='object')

In [6]:
df = df.dropna(subset = ["Longitude", "Latitude"]) # remove all listings that do not have Latitude or Longitude
df = df.dropna(subset = ["County"]) # remove all listing that do not have a County (for checklist)

# resetting indexes
df = df.reset_index(drop=True)

### creating a column that maps random colors to a room type

In [40]:
import random


In [43]:
hexadecimal

['#5472C1']

In [38]:
list(df["Room"].unique())

['Double Room',
 'Single Room',
 'Twin Room',
 'Shared Room',
 'Single & Double Room',
 'Single & Twin Room',
 'Single & Shared Room',
 'Single, Double & Shared Room',
 'Single, Double & Twin Room',
 'Double & Shared Room',
 'Double & Twin Room']

In [52]:
color_palette = dict.fromkeys(list(df["Room"].unique())
                              
for i in df["Room"].unique():
    hexadecimal = ["#"+''.join([random.choice('ABCDEF0123456789') for i in range(6)])]
    color_palette[i].append(hexadecimal)
    print(i)
    

SyntaxError: invalid syntax (Temp/ipykernel_29308/1726519082.py, line 2)

In [ ]:
# color palette
color_palette = {"Double Room":"3C486B", "Single Room": "F9D949", "Twin Room": "F45050", "Shared Room": "FF6D60", "Single & Double Room": 

In [7]:
### color based on room type 
df["color_room"] = df[df["Room"] == 3C486B

In [31]:
app = dash.Dash(__name__)

app.layout = html.Div([

    html.Div([
        html.Div([
            # checklist Bundesländer
            html.Label(children='Counties:'),
            dcc.Checklist(id='sort_by_counties', options=[{"label": str(b), "value": b} for b in sorted(df["County"].unique())],
                          value = [b for b in sorted(df["County"].unique())]),

#             # Map-Legend
#             html.Div([
#                 html.Label(children = "Legende: ", ),
#                 html.Li("B2C", className='circle', style={'background': '#2e86c1','color':'black',
#                     'list-style':'none','text-indent': '17px'}),
#                 html.Li("B2B", className= "circle", style = {"background": "#f39c12", "color":"black", "list-style": "none", "text-indent": "17px"})
#             ]),



            # html.Br(),
            # html.Label(['Website:']),
            # html.Pre(id='web_link', children=[],
            # style={'white-space': 'pre-wrap','word-break': 'break-all',
            #      'border': '1px solid black','text-align': 'center',
            #      'padding': '12px 12px 12px 12px', 'color':'blue',
            #      'margin-top': '3px'}),
        ]),

        # map
        html.Div([
            dcc.Graph(id='graph', config={'displayModeBar': False, 'scrollZoom': True},
                      style={'background': '#00FC87', 'padding-bottom': '2px', 'padding-left': '2px', 'height': '100vh'}
                        )
        ])
    ])
])

# Output of Graph
@app.callback(Output('graph', 'figure'),
              Input('sort_by_counties', 'value'))

def update_figure(chosen_county):
    df_sub = df[(df['County'].isin(chosen_county))]

    # #changes spring green for B2B to orange
    # df_sub["color"][df_sub["color"] == "#00CD66"] = "#f39c12"
    # df_sub["color"][df_sub["color"] == "#7D26CD"] = "#2e86c1"

    #hovertext
    #df_sub["Informationen"][df_sub["Informationen" == None]] = "Keine Informationen vorhanden"

    # Create figure
    locations = [go.Scattermapbox(
        lon=df_sub["Longitude"],
        lat=df_sub["Latitude"],
        mode="markers",
        marker = {"size": 10},
        unselected={"marker": {'opacity': 1}},
        selected={'marker': {'opacity': 0.5, 'size': 40}},
        hoverinfo="text",
        hovertext=df_sub[["Price_per_month",'Room']],
        # hoverlabel = {"Property_type"},
    )]

    # Return figure
    return {
        ## scattermapbox has two sections: "data" and "layout", the data section already contains the figure
        'data': locations,
        'layout': go.Layout(
            #mapbox_style = "white-bg",
            uirevision='foo',
            # preserves state of figure/map after callback activated
            clickmode= 'event+select',
            hovermode='closest',
            ## default = closest
            hoverdistance=2,
            title=dict(text="sharing properties on daft.ie", font=dict(size=30)),
            mapbox=dict(
                accesstoken=mapbox_access_token,
                bearing=0,
                #style='carto-darkmatter',
                style = "basic",
                center = dict(
                    lat = 53.350140,
                    lon = -6.266155
                ),
            pitch=40,
            zoom=11.5
            ),
        )
    }

In [32]:
# start map
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Apr/2023 17:34:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2023 17:34:31] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2023 17:34:31] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2023 17:34:31] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2023 17:34:31] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2023 17:34:31] "POST /_dash-update-component HTTP/1.1" 200 -


In [21]:
[b for b in sorted(df["County"].unique())]

['Antrim',
 'Carlow',
 'Cavan',
 'Clare',
 'Cork',
 'Donegal',
 'Down',
 'Dublin 1',
 'Dublin 10',
 'Dublin 11',
 'Dublin 12',
 'Dublin 13',
 'Dublin 14',
 'Dublin 15',
 'Dublin 16',
 'Dublin 18',
 'Dublin 2',
 'Dublin 20',
 'Dublin 22',
 'Dublin 24',
 'Dublin 3',
 'Dublin 4',
 'Dublin 5',
 'Dublin 6',
 'Dublin 7',
 'Dublin 8',
 'Dublin 9',
 'Fermanagh',
 'Galway',
 'Kerry',
 'Kildare',
 'Kilkenny',
 'Laois',
 'Limerick',
 'Longford',
 'Louth',
 'Mayo',
 'Meath',
 'Offaly',
 'Roscommon',
 'Sligo',
 'Tipperary',
 'Waterford',
 'Westmeath',
 'Wexford',
 'Wicklow']